---

# A/B Test - Problem Description

A Electronic House é um comercio online ( e-commerce ) de produtos de informática
para casas e escritórios. Os clientes podem comprar mouses, monitores, teclados,
computadores, laptops, cabos HDMI, fones de ouvido, cameras webcam, entre
outros, através de um site online e recebem os produtos no conforto de suas casas

O Diretor de Produtos Global pediu ao Head de Design que desenvolvesse uma
nova forma de finalizar a compra com cartão de crédito, sem a necessidade do
cliente preencher manualmente todas as informações do cartão e que funcionasse
em todos os países.

Depois de meses desenvolvendo esse dispositivo, o time de Desenvolvimento
Backend entregou uma solução de pagamentos, na qual **90% das informações do
formulário eram preenchido automaticamente.**

O Head de Designer gostaria de medir a efetividade do novo dispositivo de
preenchimento automático dos dados do cartão de crédito na página de vendas e
reportar os resultados ao Diretor de Produtos Global, para concluir se a nova forma
de pagamento é realmente melhor do que a antiga.

As duas páginas foram colocadas no ar e durante alguns meses e o time de FrontEnd desenvolveu uma automação que atribui um rótulo para cada cliente,
mostrando qual a página de vendas aquele determinado cliente estava visualizando.
Todos esses dados foram armazenados em um banco de dados e podem ser
acessados pelos times da Electronic House.

Depois de alguns meses de experimento, o time de Designers da Electronic House,
precisa avaliar os resultados do experimento realizado, a fim de concluir qual era a
forma de pagamento mais eficaz.

Porém, o time não possui as habilidades necessárias para avaliar os dados e
concluir o experimento. Nesse contexto, você foi contratado como Cientista de
Dados para ajudar o time de Designers a **validar a efetividade do novo meio de
pagamento, com mais confiança e rigidez na análise.**

O entregável do seu trabalho é o seguinte:
1. Qual a melhor forma de pagamento: Preenchimento Manual ou Automático do
formulário de dados do cartão de crédito?

# 0.0 Imports

In [11]:
import pandas as pd
import numpy as np

from statsmodels.stats import api as sms

## 0.1 Load Data

In [4]:
path = 'C:/Users/edils/repos/teste_ab/data/'

df_raw = pd.read_csv(path + 'ab_testing.csv')

In [5]:
df_raw.head()

,uid,country,gender,spent,purchases,date,group,device
0,11115722,MEX,F,1595,5,2016-03-08,GRP B,I
1,11122053,USA,M,498,2,2017-07-14,GRP B,I
2,11128688,USA,F,2394,6,2017-09-17,GRP A,I
3,11130578,USA,F,1197,3,2017-11-30,GRP A,I
4,11130759,ESP,M,1297,3,2018-01-10,GRP B,A


In [6]:
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45883 entries, 0 to 45882
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   uid        45883 non-null  int64 
 1   country    45883 non-null  object
 2   gender     45883 non-null  object
 3   spent      45883 non-null  int64 
 4   purchases  45883 non-null  int64 
 5   date       45883 non-null  object
 6   group      45883 non-null  object
 7   device     45883 non-null  object
dtypes: int64(3), object(5)
memory usage: 2.8+ MB


# 1.0 Design do Experimento

In [7]:
# Objetivo: Definir se o preenchimento automático é melhor que que o manual
# Métrica: GMV médio

# Grupo A = Preenchimento automático
# Grupo B = Preenchimento Manual

## 1.1 Hipóteses do Experimento

In [ ]:
# H0 : média de gastos(GMV) da página A = média de gastos(GMV) da página B 
# H1 : média de gastos(GMV) da página A <> média de gastos(GMV) da página B

## 1.2 Tamanho da Amostra

In [21]:
#Effect size

#init_metric = Média Sales ou Média GMV
#end_metric = metric lift (5% sobre a metrica atual)
#std_metric = Desvio padrão das Sales ou GMV

#This values are from mean and std from Brazil
init_metric = 1902.86
end_metric = init_metric * 1.05

std_metric = 1398.88

effect_size = (end_metric - init_metric) / std_metric

sample_size = sms.tt_ind_solve_power(
    effect_size = effect_size,
    alpha = 0.05,
    power = 0.80
)

print(f"Número de Amostras: {np.ceil(sample_size).astype(int)}")

Número de Amostras: 3395


# 2.0 Preparação dos Dados

In [12]:
df2 = df_raw.copy()

In [16]:
# Use Data only from Brazil
df2_br = df2.loc[df2['country'] == 'BRA',:]

# mean and std
df2_br['spent'].agg(['mean','std'])

mean    1902.860251
std     1398.886279
Name: spent, dtype: float64

In [17]:
df2_br

,uid,country,gender,spent,purchases,date,group,device
12,11143138,BRA,M,2592,8,2015-05-20,GRP B,A
15,11148343,BRA,M,299,1,2016-09-04,GRP B,A
31,11172970,BRA,M,1797,3,2017-07-05,GRP B,I
36,11177332,BRA,M,1994,6,2017-01-06,GRP B,I
37,11179295,BRA,M,1198,2,2017-12-19,GRP B,A
...,...,...,...,...,...,...,...,...
45852,99940432,BRA,M,897,3,2017-09-13,GRP A,I
45860,99955317,BRA,M,698,2,2017-10-17,GRP A,A
45871,99977039,BRA,M,1495,5,2016-05-06,GRP B,A
45877,99991883,BRA,M,2995,5,2017-06-22,GRP B,A


# 3.0 Testes de Hióteses

# 4.0 Conclusão